Gerardo Rivas Alvarez.

### Ejercicio 1

En clase vimos que la función `bits` permite ver la representación binaria de números de punto flotante (`Float64`) o de enteros (e.g., `Int64`). En particular, para números de punto flotante, el primer bit es el signo, los siguiente 11 son el *exponente* y el resto es la representación binaria (sin el 1 del principio), del número en cuestión.

La pregunta es ¿qué significa el exponente?

```julia
bits(0.1)
"0111111111110000000000000000000000000000000000000000000000000000"
```

Tiene que ver con la "notación científica" de la computadora. Para ella un número flotante se puede escribir como $x=q\times 2^{m}$. Y el exponene de la representación binaria tiene que ver justo con $m$.

### Ejercicio 2

- ¿Cuál es el número más chico que sumado (o restado) a 1.0 me da algo distinto de 1.0?
(Este número se bautiza como el "épsilon de la máquina")

- ¿Cuál es el número más chico que sumado (o restado) a 10.0 me da algo distinto de 10.0?


Aproximé la épsilon de la máquina para 1 y para 10 por medio de potencias de e, la base del logaritmo natural. 
A prueba y error encontre que la potencia que jala para ambos está entre -40 y -30. Entonces hice un código que busca el exponente para el cual 1+e^(-i) = 1. El código no es más que un ciclo for que recorre el intervalo [30,40] con pasos de 0.0001 y busca el mayor número que al sumarlo a 1.0 (10.0) devuelva 1.0 (10.0), sea $\epsilon /2$ este número. Luego, el resultado que doy es $\epsilon$. Este $\epsilon$  **no** es el número positivo más pequeño que al sumarlo a 1.0 (10.0) me da algo diferente de 1.0 (10.0). De hecho ni siquiera lo es `eps(1.0)`, la muestra está abajo, en la primer línea de código después de esto. $\epsilon$ sólo es una **aproximáción** a la "épsilon de la máquina" y da una idea del orden de magnitud de esta.

In [7]:
1.0 + eps(1.0)*0.6

1.0000000000000002

In [10]:
for i in 30:0.0001:40
    if 1.0 + e^(-i) == 1.0
        println("Una aproximación para la épsilon de 1.0 (uno) es ", 2*e^(-i))
        break
    end
end


Una aproximación para la épsilon de 1.0 (uno) es 2.2202252805583935e-16


In [6]:
for i in 30:0.0001:40
    if 10 + e^(-i) == 10
        println("Una aproximación para la épsilon de 10 (diez) es ", 2*e^(-i))
        break
    end
end

Una aproximación para la épsilon de 10 (diez) es 1.7762840599939335e-15


### Ejercicio 3

Define una functión llamada `derivada1` que implemente el cálculo de:

$$
\frac{\Delta f(x)}{\Delta x}(x_0) = \frac{f(x_0+h)-f(x_0)}{h}
$$

Obviamente esta función es una aproximación *finita* de la derivada. La pregunta es ¿qué tan buena es esta aproximación, pensando en términos de $h$?

- La primer sub-pregunta es ¿cómo se responde este tipo de pregunta?
- Uno tiene que tomar un caso concreto (para $f(x_0,h)$); sugiero empezar con un polinomio con coeficientes enteros, y después hacer el mismo tipo de pruebas con una función trascendental.
- ¿Hay algo *sorprendente* en el comportamiento de `derivada1` como función de $h$? ¿A qué lo puedes atribuir?
- ¿Puedes hacer que la función `derivada1` funcione para funciones más o menos arbitrarias, que el usuario define?
- ¿Puedes explicar de alguna manera los resultados que has obtenido?

In [2]:
#Se define la función derivada que calcula la derivada de una función con la definicón usual.
function derivada1(funcion,x_0,h)
    (funcion(x_0+h)-funcion(x_0))/h
end

derivada1 (generic function with 1 method)

In [24]:
g(x)=x^67
derivada1(g,1.0,0.000000000000001)

74.38494264988549

In [28]:
f(x)=sin(x)
derivada1(f,0.0,0.0000001)

0.9999999999999983

Este método se basa en la definición formal de la derivada. Y haciendo el desarrollo en serie de Taylor de $f(x)$ alrededor de $x_0$ se observa que el error va como $h$. Es un error que siempre se tendrá. Pero aquí observé gracias a las observaciones de Uriel, que para los polinomios, si se elige $h \sim 10^{-16}$, el error supera horriblemente a $h$. Esto lo atribuyo a la representación finita de los flotantes en la computadora, un error que no contempla el método numérico.

Si la función que define el usuario no es "continua" en el siguiente sentido, una relación obtenida empíricamente. En este caso nada asegura que $f(x+h)$ exista si quiera. En ese caso no se me ocurre que pueda hacer con `derivada1`.

Como conclusion de este ejercicio, este método se basa en la definición formal de la derivada. Tiene un error numérico intrínseco del orden de mágnitud de $h$ y otro error computacional debido a la represantación finita de los reales.

### Ejercicio 4

Este ejercicio es equivalente al anterior, pero considerando la función `derivada2` que implemente el cálculo de:

$$
\frac{\Delta f(x)}{\Delta x}(x_0) = \frac{f(x_0+h)-f(x_0-h)}{2h}
$$

¿Puedes explicar por qué la convergencia es mejor en este caso?


In [31]:
function derivada2(f, x_0, h)
    (f(x_0 + h)-f(x_0 - h))/2h
    
end

derivada2 (generic function with 1 method)

In [55]:
f(x)=sin(x)

#derivada1(f,1.,0.1)
derivada2(f,0.0,0.0000001)

0.9999999999999983

In [58]:
g(x)=x^42
derivada2(g,1.0,0.0000000000000001)

23.314683517128287

Si se desarrolla $f(x_0 + h)$ y $f(x_0 - h)$ en serie de Taylor y se hace la diferencia, se anula el error lineal en $h$ y el siguiente error apreciable va como $h^2$. Este error es un error numérico, pero evaluando la función `derivada2` con $h\sim 10^{-16}$ se observa que el error computacional supera al error numérico, en el caso de polinomios.

### Ejercicio 5

Manda el notebook con tus resultados haciendo un pull-request al repositorio del curso. Para hacer esto:
- Haz un `fork` del repositorio del curso a tu propia cuenta en GitHub
- Una vez que tengas resuelto el notebook, súbelo a **tu** repositorio (con `git push`), en la carpeta "tareas/usuario_github/nombre_notebook"
- Desde tu repositorio, haz un "Pull Request". El *pull request* sólo se aceptará si es legible en el sentido más amplio de la palabra y, obviamente, si es correcto. Esto significa que debe ser *entendible*, a partir de lo que ustedes escriben, incluso para gente que no está en el curso. (Como GitHub es público, piensen fríamente que esto puede ser leido por *muuucha* gente allá afuera.) Si el trabajo es en colaboración, háganlo saber en el propio notebook (cosa que se debe notar en el historial de lo que envíen). Finalmente, como código ético, incluyan cualquier fuente de inspiración, incluso cuando se trate del notebook de otro compañero/equipo del curso.

##Fuentes

- El notebook de mi compañero Bertoni.
- Métodos numéricos y computación. Ward Cheney, David Kincaid. CENGAGE learning.